In [1]:
import pandas as pd
import numpy as np
from pickle import load
import pickle

In [2]:
train_data = open('train_data.pkl', 'rb')
train_dict = load(train_data)
train_data.close()

In [3]:
disease_list = list()
for key in train_dict.keys():
    for hadm in train_dict[key].keys():
        disease_list.append(' '.join(train_dict[key][hadm][0]))

In [4]:
disease_list[:2]

['03842 78552 5849 591 5990 42731 4280 1105 99592 29420 24290 4019 71590 5931',
 'V3001 V053 V290']

In [7]:
from keras.preprocessing.text import Tokenizer

tokeniser = Tokenizer()
tokeniser.fit_on_texts(disease_list)

In [10]:
vocab_size = len(tokeniser.word_index) + 1

In [11]:
sequences_list = list()
for line in disease_list:
    seq = tokeniser.texts_to_sequences([line])[0]
    for i in range(1, len(seq)):
        sequence = seq[:i+1]
        sequences_list.append(sequence)

In [21]:
from keras.preprocessing.sequence import pad_sequences

sequences_list  = pad_sequences(sequences_list, maxlen=50, padding='pre')

In [23]:
from numpy import array

sequences_list = array(sequences_list)

In [31]:
from keras.utils import to_categorical

X, y = sequences_list[:, :-1], sequences_list[:, -1]
# y = to_categorical(y, num_classes=vocab_size)

In [39]:
len(X[0])

49

In [42]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras import metrics

model = Sequential()
model.add(Embedding(vocab_size, 256, input_length=49))
model.add(LSTM(512))
model.add(Dense(vocab_size, activation='softmax'))

print(model.summary())

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 49, 256)           1744896   
_________________________________________________________________
lstm_6 (LSTM)                (None, 512)               1574912   
_________________________________________________________________
dense_6 (Dense)              (None, 6816)              3496608   
Total params: 6,816,416
Trainable params: 6,816,416
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
 33472/533318 [>.............................] - ETA: 5:16:21 - loss: 6.3372 - sparse_categorical_accuracy: 0.0522

KeyboardInterrupt: 